In [ ]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import math
import random
import numpy as np

In [ ]:
BATCH_SIZE = 128

# Load data

In [ ]:
path_train = "data/binarized_mnist_train.amat"
path_valid = "data/binarized_mnist_valid.amat"
path_test = "data/binarized_mnist_test.amat"

In [ ]:
def data_loader(path):
    with open(path_train) as file:
        x = [
            np.array(list(map(int, img.split()))).reshape(1, 28, 28)
            for img in file.readlines()
        ]
    return torch.from_numpy(np.asarray(x)).float()

In [ ]:
train = torch.utils.data.TensorDataset(data_loader(path_train))
trainloader = torch.utils.data.DataLoader(
    train, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
valid = torch.utils.data.TensorDataset(data_loader(path_train))
validloader = torch.utils.data.DataLoader(
    train, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
test = torch.utils.data.TensorDataset(data_loader(path_train))
testloader = torch.utils.data.DataLoader(
    train, batch_size=BATCH_SIZE, shuffle=False)

# Train a VAE (10pts)

In [ ]:
class VAE(nn.Module):
    def __init__(self, dim=100):
        super(VAE, self).__init__()

        self.dim = dim

        #ENCODER LAYERS
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=3), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=5),
            nn.ELU())
        self.hidden_1 = nn.Linear(256, 200)

        #DECODER LAYERS
        self.hidden_2 = nn.Linear(100, 256)
        self.decoder = nn.Sequential(
            nn.ELU(),
            nn.Conv2d(
                in_channels=256, out_channels=64, kernel_size=5, padding=4),
            nn.ELU(), nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(
                in_channels=64, out_channels=32, kernel_size=3, padding=2),
            nn.ELU(), nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(
                in_channels=32, out_channels=16, kernel_size=3, padding=2),
            nn.ELU(),
            nn.Conv2d(
                in_channels=16, out_channels=1, kernel_size=3, padding=2),
            nn.Sigmoid())

    def encode(self, x):
        q_params = self.hidden_1(self.encoder(x).view(x.shape[0], 256))
        mu = q_params[:, :self.dim]
        logvar = q_params[:, self.dim:]
        return mu, logvar

    def decode(self, z, x):
        return self.decoder(self.hidden_2(z).view(x.shape[0], 256, 1, 1))

    def sample(self, mu, logvar, x):
        eps = torch.randn(x.shape[0], self.dim).to(device)
        return mu + eps * (0.5 * logvar).exp()

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.sample(mu, logvar, x)
        x_hat = self.decode(z, x)
        return x_hat, mu, logvar

    def criterion(self, x_hat, x, mu, logvar):
        # BCE = -log(p) because gradient descent and not ascent
        BCE = F.binary_cross_entropy(x_hat, x, reduction='sum') 
        KLD = 0.5 * torch.sum(-1 - logvar + mu.pow(2) + logvar.exp())
        # criterion returns -ELBO !
        return (BCE + KLD) / x.shape[0]

    def evaluate(self, loader):
        with torch.no_grad():
            loss = 0
            for i, data in enumerate(loader, 1):
                # get the inputs
                x = data[0].to(device)
                x_hat, mu, logvar = self.forward(x)
                loss += self.criterion(x_hat, x, mu, logvar).item()
        return loss / len(loader)

In [ ]:
# create model and move it to device
model = VAE()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Let's use {}".format(device))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        x = data[0].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        x_hat, mu, logvar = model.forward(x)

        loss = model.criterion(x_hat, x, mu, logvar)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # print statistics
    train_loss = running_loss / i
    val_loss = model.evaluate(validloader)
    print('epoch %2d: loss: %8.3f    val_loss: %8.3f    val_ELBO: %8.3f' %
          (epoch + 1, train_loss, val_loss, -val_loss))

# Evaluating log-likelihood with Variational Autoencoders (20pts)

In [ ]:
def importance_sampling_estimation(model, data, K=200, D=784, L=100, M=10):
    estimates = []
    for _ in range(M):
        x = random.choice(list(data))[0].view(1, 1, 28, 28).to(device)
        mu, logvar = model.encode(x)
        p = 0
        for _ in range(K):
            z = model.sample(mu, logvar, x)
            x_hat = model.decode(z, x)
            p += model.criterion(x_hat, x, mu, logvar)
        estimates.append(p / K)
    return estimates

In [ ]:
importance_sampling_estimation(model, test)